In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, r2_score, mean_squared_error

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from category_encoders import MEstimateEncoder, TargetEncoder
from sklearn.inspection import permutation_importance

import seaborn as sns
import shap

import lightgbm as lgb

import sys
sys.path.append('../lightgbm')

C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_order(al

In [2]:
# Train datasets
invesment_train_path = 'processed/processed_train_inv.csv'
owner_train_path = 'processed/processed_train_own.csv'

# Test
investment_test_path = 'processed/processed_test_inv.csv'
owner_test_path = 'processed/processed_test_own.csv'

# Feats
# investment_feat_path = './best_feats/Inv_Onehot_only_mean_NANs_modified_prices.csv'
# owner_feat_path = './best_feats/Own_Onehot_only_mean_NANs_modified_prices.csv'

output_path = './output/lgbm_split_pred.csv'

In [3]:
processed_inv_train_df = pd.read_csv(invesment_train_path)
X = processed_inv_train_df.drop(['price_doc'], axis=1)
y = processed_inv_train_df['price_doc']

processed_inv_train_df

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,area_m,raion_popul,...,big_road1_1line_yes,railroad_1line_no,railroad_1line_yes,material_1.0,material_2.0,material_4.0,material_5.0,material_6.0,material_nan,price_doc
0,43,27.0,4.0,12.184338,1974,1.967868,8.124853,2.480988,6.407578e+06,155572,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.668660e+06
1,34,19.0,3.0,12.184338,1974,1.967868,8.124853,2.480988,9.589337e+06,115352,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.814010e+06
2,43,29.0,2.0,12.184338,1974,1.967868,8.124853,2.480988,4.808270e+06,101708,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.523310e+06
3,89,50.0,9.0,12.184338,1974,1.967868,8.124853,2.480988,1.258354e+07,178473,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.269391e+07
4,77,77.0,4.0,12.184338,1974,1.967868,8.124853,2.480988,8.398461e+06,108171,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.582519e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19441,56,29.0,13.0,14.000000,2001,2.000000,11.000000,3.000000,9.249237e+06,156377,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.162801e+07
19442,44,27.0,7.0,9.000000,1975,2.000000,6.000000,3.000000,1.005305e+07,175518,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.170610e+06
19443,86,59.0,3.0,9.000000,1935,4.000000,10.000000,3.000000,7.307411e+06,75377,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.422501e+07
19444,64,32.0,5.0,15.000000,2003,2.000000,11.000000,2.000000,6.050065e+06,78616,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.308151e+07


In [4]:
X_train, X_val, y_train, y_val = train_test_split(processed_inv_train_df.drop(columns=['price_doc']),
                                                 processed_inv_train_df['price_doc'], test_size=0.15, random_state=42)

In [5]:
from sklearn.metrics import mean_squared_error
import optuna

def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "n_estimators": trial.suggest_int("n_estimators", 800, 1200),
#         "boosting": trial.suggest_categorical("boosting", ["gbdt", "rf", "dart"]),
        "lambda_l2": trial.suggest_float("lambda_l2", 0, 10),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0, 1),
        "bagging_freq": 1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1),
        "max_depth": trial.suggest_int("max_depth", 1, 50),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train, verbose=False)
    predictions = model.predict(X_val)
    rmse = mean_squared_error(y_val, predictions, squared=False)
    return rmse

In [6]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

[I 2023-10-30 19:18:46,022] A new study created in memory with name: no-name-cac2f260-5d79-4550-bf2d-d343bf3959cc
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.938091862962235, colsample_bytree=0.23134149144796534 will be ignored. Current value: feature_fraction=0.938091862962235
[LightGBM] [Warning] min_data_in_leaf is set=80, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=80
[LightGBM] [Warning] bagging_fraction is set=0.13910793455675197, subsample=0.3936949176994204 will be ignored. Current value: bagging_fraction=0.13910793455675197
[LightGBM] [Warning] lambda_l2 is set=3.9824559370122836, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.9824559370122836
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-30 19:18:49,677] Trial 0 finished with value: 3364431.7653503967 and parameters: {'n_estimators': 1128, 'lambda_l2': 3.9824559370122836, 'bagging_fraction': 0.13910793455675197, 'num_leaves': 691, 'feature_fraction': 0.938091862962235, 'max_depth': 21, 'learning_rate': 0.009023639134607136, 'subsample': 0.3936949176994204, 'colsample_bytree': 0.23134149144796534, 'min_data_in_leaf': 80}. Best is trial 0 with value: 3364431.7653503967.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.9988628690312592, colsample_bytree=0.3151460793789506 will be ignored. Current value: feature_fraction=0.9988628690312592
[LightGBM] [Warning] min_data_in_leaf is set=18, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=18
[LightGBM] [Warning] bagging_fraction is set=0.512877778687155, subsample=0.26126814374373 will be ignored. Current value: bagging_fraction=0.512877778687155
[LightGBM] [Warning] lambda_l2 is set=9.161204202980027, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.161204202980027
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-30 19:18:50,995] Trial 1 finished with value: 2987602.7076837807 and parameters: {'n_estimators': 856, 'lambda_l2': 9.161204202980027, 'bagging_fraction': 0.512877778687155, 'num_leaves': 906, 'feature_fraction': 0.9988628690312592, 'max_depth': 2, 'learning_rate': 0.07137880477899301, 'subsample': 0.26126814374373, 'colsample_bytree': 0.3151460793789506, 'min_data_in_leaf': 18}. Best is trial 1 with value: 2987602.7076837807.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


...

[I 2023-10-30 19:26:31,963] Trial 47 finished with value: 2971790.2978260084 and parameters: {'n_estimators': 1089, 'lambda_l2': 6.890894607571956, 'bagging_fraction': 0.5266951585585466, 'num_leaves': 511, 'feature_fraction': 0.9267029088156988, 'max_depth': 7, 'learning_rate': 0.0275530681366213, 'subsample': 0.3023593319567359, 'colsample_bytree': 0.959079007817435, 'min_data_in_leaf': 59}. Best is trial 45 with value: 2952960.6177072064.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.9242956520276048, colsample_bytree=0.9695617569026996 will be ignored. Current value: feature_fraction=0.9242956520276048
[LightGBM] [Warning] min_data_in_leaf is set=60, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=60
[LightGBM] [Warning] bagging_fraction is set=0.6407743434274321, subsample=0.2924535733961824 will be ignored. Current value: bagging_fraction=0.6407743434274321
[LightGBM] [Warning] lambda_l2 is set=7.0088410792581275, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.0088410792581275
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-30 19:26:45,227] Trial 48 finished with value: 3065216.81594022 and parameters: {'n_estimators': 1066, 'lambda_l2': 7.0088410792581275, 'bagging_fraction': 0.6407743434274321, 'num_leaves': 504, 'feature_fraction': 0.9242956520276048, 'max_depth': 19, 'learning_rate': 0.02462177181724547, 'subsample': 0.2924535733961824, 'colsample_bytree': 0.9695617569026996, 'min_data_in_leaf': 60}. Best is trial 45 with value: 2952960.6177072064.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.889790105607426, colsample_bytree=0.9972198433122792 will be ignored. Current value: feature_fraction=0.889790105607426
[LightGBM] [Warning] min_data_in_leaf is set=53, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=53
[LightGBM] [Warning] bagging_fraction is set=0.5457858489699411, subsample=0.30996665641119436 will be ignored. Current value: bagging_fraction=0.5457858489699411
[LightGBM] [Warning] lambda_l2 is set=5.912812371287156, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.912812371287156
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-30 19:26:55,988] Trial 49 finished with value: 3020235.996688494 and parameters: {'n_estimators': 1092, 'lambda_l2': 5.912812371287156, 'bagging_fraction': 0.5457858489699411, 'num_leaves': 322, 'feature_fraction': 0.889790105607426, 'max_depth': 15, 'learning_rate': 0.013262003662218774, 'subsample': 0.30996665641119436, 'colsample_bytree': 0.9972198433122792, 'min_data_in_leaf': 53}. Best is trial 45 with value: 2952960.6177072064.


In [10]:
inv_params = study.best_params
inv_params["objective"] = "regression"
inv_params["metric"] = "rmse"
inv_params["verbosity"] = -1

model = lgb.LGBMRegressor(**inv_params)

model.fit(X, y)

[LightGBM] [Warning] feature_fraction is set=0.9468185243604629, colsample_bytree=0.9685869661027242 will be ignored. Current value: feature_fraction=0.9468185243604629
[LightGBM] [Warning] min_data_in_leaf is set=42, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=42
[LightGBM] [Warning] bagging_fraction is set=0.578043267587271, subsample=0.2807606573380648 will be ignored. Current value: bagging_fraction=0.578043267587271
[LightGBM] [Warning] lambda_l2 is set=7.389294926795998, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.389294926795998


LGBMRegressor(bagging_fraction=0.578043267587271,
              colsample_bytree=0.9685869661027242,
              feature_fraction=0.9468185243604629, lambda_l2=7.389294926795998,
              learning_rate=0.024152436863543725, max_depth=7, metric='rmse',
              min_data_in_leaf=42, n_estimators=1093, num_leaves=903,
              objective='regression', subsample=0.2807606573380648,
              verbosity=-1)

In [12]:
# test_df = pd.read_csv(test_df_path)
processed_inv_test_df = pd.read_csv(investment_test_path)
processed_inv_test_df_copy = processed_inv_test_df.copy()
processed_inv_test_df = processed_inv_test_df.drop(['id'], axis=1)
processed_inv_test_df

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,area_m,raion_popul,...,big_road1_1line_no,big_road1_1line_yes,railroad_1line_no,railroad_1line_yes,material_1.0,material_2.0,material_4.0,material_5.0,material_6.0,material_nan
0,39.0,20.7,2,9,1998.0,1,8.9,3.000000,2.615514e+07,178264,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,40.5,25.1,3,5,1960.0,2,4.8,2.000000,9.946335e+06,139322,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,45.4,28.5,9,12,1972.0,2,6.0,2.000000,1.531990e+07,21040,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,39.8,18.9,4,17,2013.0,1,9.9,3.000000,1.139168e+07,19940,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,94.5,61.9,2,12,2000.0,4,10.3,2.480988,5.704502e+06,106445,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,39.6,18.9,22,25,2014.0,1,9.1,2.000000,1.050803e+07,81887,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4994,54.6,32.2,16,16,2000.0,2,9.3,3.000000,1.175477e+07,247469,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4995,52.2,31.8,10,12,1973.0,2,9.1,2.000000,7.811375e+06,153248,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4996,34.8,19.8,8,9,1977.0,1,6.4,2.000000,7.128794e+06,145576,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [14]:
inv_pred = model.predict(processed_inv_test_df)
inv_prediction_df = pd.DataFrame({
    'id': processed_inv_test_df_copy['id'],
    'price_doc': inv_pred
})

In [15]:
processed_owner_train_df = pd.read_csv(owner_train_path)
X = processed_owner_train_df.drop(['price_doc'], axis=1)
y = processed_owner_train_df['price_doc']

processed_owner_train_df

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,area_m,raion_popul,...,railroad_1line_no,railroad_1line_yes,material_1.0,material_2.0,material_3.0,material_4.0,material_5.0,material_6.0,material_nan,price_doc
0,73,38.000000,15.0,13.257777,1493,1.801699,3.183089,1.075835,8.659075e+06,104410,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.998524e+06
1,117,108.000000,20.0,13.257777,1493,1.801699,3.183089,1.075835,7.750778e+06,76284,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.366640e+07
2,83,83.000000,18.0,13.257777,1493,1.801699,3.183089,1.075835,8.659075e+06,104410,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.102727e+07
3,73,49.632838,14.0,13.257777,1493,1.801699,3.183089,1.075835,8.659075e+06,104410,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.787838e+06
4,70,36.000000,9.0,13.257777,1493,1.801699,3.183089,1.075835,8.659075e+06,104410,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.372372e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11018,51,49.632838,4.0,17.000000,1493,2.000000,1.000000,1.000000,1.132409e+07,113897,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5.599757e+06
11019,47,30.000000,23.0,25.000000,2016,1.000000,10.000000,1.000000,9.629358e+06,89467,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.021721e+07
11020,56,51.000000,19.0,19.000000,2017,2.000000,1.000000,1.000000,4.441296e+06,60315,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,9.943898e+06
11021,52,49.632838,13.0,17.000000,1493,2.000000,1.000000,1.000000,7.307411e+06,75377,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.541297e+06


In [17]:
X_train, X_val, y_train, y_val = train_test_split(processed_owner_train_df.drop(columns=['price_doc']),
                                                 processed_owner_train_df['price_doc'], test_size=0.15, random_state=42)

In [18]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

[I 2023-10-30 19:33:29,915] A new study created in memory with name: no-name-fa8f84eb-5f8d-47e4-aa91-1045e25de49a
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.5863676320264337, colsample_bytree=0.32469576813080003 will be ignored. Current value: feature_fraction=0.5863676320264337
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] bagging_fraction is set=0.7838715639496036, subsample=0.9585337302231216 will be ignored. Current value: bagging_fraction=0.7838715639496036
[LightGBM] [Warning] lambda_l2 is set=8.836713397634348, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.836713397634348
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-30 19:33:34,921] Trial 0 finished with value: 2134587.034605441 and parameters: {'n_estimators': 1184, 'lambda_l2': 8.836713397634348, 'bagging_fraction': 0.7838715639496036, 'num_leaves': 478, 'feature_fraction': 0.5863676320264337, 'max_depth': 37, 'learning_rate': 0.0014476791201077511, 'subsample': 0.9585337302231216, 'colsample_bytree': 0.32469576813080003, 'min_data_in_leaf': 86}. Best is trial 0 with value: 2134587.034605441.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.8882517178789922, colsample_bytree=0.6045622986330635 will be ignored. Current value: feature_fraction=0.8882517178789922
[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Warning] bagging_fraction is set=0.11616196389840683, subsample=0.9643279669440986 will be ignored. Current value: bagging_fraction=0.11616196389840683
[LightGBM] [Warning] lambda_l2 is set=1.282895085827792, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.282895085827792
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-30 19:33:36,480] Trial 1 finished with value: 2332995.4331506793 and parameters: {'n_estimators': 1105, 'lambda_l2': 1.282895085827792, 'bagging_fraction': 0.11616196389840683, 'num_leaves': 232, 'feature_fraction': 0.8882517178789922, 'max_depth': 9, 'learning_rate': 0.0055230289886807555, 'subsample': 0.9643279669440986, 'colsample_bytree': 0.6045622986330635, 'min_data_in_leaf': 88}. Best is trial 0 with value: 2134587.034605441.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


...

[I 2023-10-30 19:42:49,046] Trial 47 finished with value: 1328194.6880066844 and parameters: {'n_estimators': 1152, 'lambda_l2': 4.34524461968002, 'bagging_fraction': 0.4364762916518884, 'num_leaves': 579, 'feature_fraction': 0.8813454308103872, 'max_depth': 6, 'learning_rate': 0.01656157100947154, 'subsample': 0.6688506796634324, 'colsample_bytree': 0.49279095153484465, 'min_data_in_leaf': 25}. Best is trial 31 with value: 1197787.7956579023.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.9217311631552456, colsample_bytree=0.3856556934062084 will be ignored. Current value: feature_fraction=0.9217311631552456
[LightGBM] [Warning] min_data_in_leaf is set=18, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=18
[LightGBM] [Warning] bagging_fraction is set=0.4827124013717743, subsample=0.5774846532140507 will be ignored. Current value: bagging_fraction=0.4827124013717743
[LightGBM] [Warning] lambda_l2 is set=4.845474882962658, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.845474882962658
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-30 19:43:06,537] Trial 48 finished with value: 1241565.830604156 and parameters: {'n_estimators': 1181, 'lambda_l2': 4.845474882962658, 'bagging_fraction': 0.4827124013717743, 'num_leaves': 236, 'feature_fraction': 0.9217311631552456, 'max_depth': 48, 'learning_rate': 0.021260860642279257, 'subsample': 0.5774846532140507, 'colsample_bytree': 0.3856556934062084, 'min_data_in_leaf': 18}. Best is trial 31 with value: 1197787.7956579023.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.9027184066086741, colsample_bytree=0.631806171373282 will be ignored. Current value: feature_fraction=0.9027184066086741
[LightGBM] [Warning] min_data_in_leaf is set=79, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=79
[LightGBM] [Warning] bagging_fraction is set=0.5137812721797204, subsample=0.6187658462087475 will be ignored. Current value: bagging_fraction=0.5137812721797204
[LightGBM] [Warning] lambda_l2 is set=7.379078628327385, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.379078628327385
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-30 19:43:11,340] Trial 49 finished with value: 1551658.7602581284 and parameters: {'n_estimators': 1129, 'lambda_l2': 7.379078628327385, 'bagging_fraction': 0.5137812721797204, 'num_leaves': 501, 'feature_fraction': 0.9027184066086741, 'max_depth': 21, 'learning_rate': 0.008509250164538615, 'subsample': 0.6187658462087475, 'colsample_bytree': 0.631806171373282, 'min_data_in_leaf': 79}. Best is trial 31 with value: 1197787.7956579023.


In [19]:
owner_params = study.best_params
owner_params["objective"] = "regression"
owner_params["metric"] = "rmse"
owner_params["verbosity"] = -1

model = lgb.LGBMRegressor(**owner_params)

model.fit(X, y)

[LightGBM] [Warning] feature_fraction is set=0.8513281526025094, colsample_bytree=0.4509899255153016 will be ignored. Current value: feature_fraction=0.8513281526025094
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_fraction is set=0.49247225264379724, subsample=0.6758101493215637 will be ignored. Current value: bagging_fraction=0.49247225264379724
[LightGBM] [Warning] lambda_l2 is set=5.549370556597469, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.549370556597469


LGBMRegressor(bagging_fraction=0.49247225264379724,
              colsample_bytree=0.4509899255153016,
              feature_fraction=0.8513281526025094, lambda_l2=5.549370556597469,
              learning_rate=0.013130469975995469, max_depth=45, metric='rmse',
              min_data_in_leaf=10, n_estimators=1164, num_leaves=495,
              objective='regression', subsample=0.6758101493215637,
              verbosity=-1)

In [20]:
processed_owner_test_df = pd.read_csv(owner_test_path)
processed_owner_test_df_copy = processed_owner_test_df.copy()
processed_owner_test_df = processed_owner_test_df.drop(['id'], axis=1)
processed_owner_test_df

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,area_m,raion_popul,...,big_road1_1line_yes,railroad_1line_no,railroad_1line_yes,material_1.0,material_2.0,material_3.0,material_4.0,material_5.0,material_6.0,material_nan
0,79.20,49.632838,8,17,0.000000,3,1.0,1.000000,2.553630e+07,4001,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,62.80,36.000000,17,17,2016.000000,2,62.8,3.000000,2.149409e+07,7122,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,40.00,40.000000,17,17,0.000000,1,1.0,1.000000,2.553630e+07,4001,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,48.43,49.632838,21,1,2015.000000,1,1.0,1.000000,9.629358e+06,89467,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,38.80,49.632838,15,17,1493.225347,1,1.0,1.000000,1.132409e+07,113897,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2626,23.56,1.000000,1,1,1.000000,1,1.0,1.000000,5.032753e+07,5740,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2627,41.68,1.000000,13,17,1493.225347,1,1.0,1.000000,7.878765e+06,32241,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2628,50.28,1.000000,3,17,1493.225347,1,1.0,1.000000,4.036700e+07,21819,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2629,54.09,49.632838,14,0,1493.225347,2,0.0,1.075835,5.299528e+07,13890,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
owner_pred = model.predict(processed_owner_test_df)
owner_prediction_df = pd.DataFrame({
    'id': processed_owner_test_df_copy['id'],
    'price_doc': owner_pred
})

In [25]:
overall_preds = pd.concat([inv_prediction_df, owner_prediction_df])

overall_preds.sort_values(by=['id'], inplace=True)

overall_preds

,id,price_doc
0,30474,5.644533e+06
0,30475,8.368661e+06
1,30476,5.547279e+06
1,30477,6.317920e+06
2,30478,4.642308e+06
...,...,...
4995,38131,7.997103e+06
2629,38132,4.888978e+06
2630,38133,4.045295e+06
4996,38134,5.402882e+06


In [24]:
overall_preds.to_csv(output_path, index=False)